# 🚢 EPA Titanic — SGDClassifier V1 (Modelos lineales estocásticos)

En este notebook probaremos el **SGDClassifier** de scikit-learn aplicado al clásico problema del Titanic.  

## 🔎 ¿Qué es el SGDClassifier?

El **SGDClassifier** es un clasificador lineal que entrena el modelo mediante **descenso de gradiente estocástico (SGD, Stochastic Gradient Descent)**.  
A diferencia de `LogisticRegression` o `LinearSVC`, que usan optimizadores más "exactos", este modelo ajusta los parámetros en forma **iterativa y aproximada**, lo que lo hace:

- ⚡ **Muy eficiente** en datasets grandes y dispersos.  
- 🔄 Compatible con distintos tipos de funciones de pérdida (`loss`), lo que le da flexibilidad:  
  - `loss="log_loss"` → equivalente a regresión logística.  
  - `loss="hinge"` → equivalente a SVM lineal.  
  - `loss="modified_huber"`, `perceptron`, etc.  
- 🎛️ Permite ajustar la regularización: `penalty = l1`, `l2`, o `elasticnet`.  
- 📉 Actualiza los parámetros en **mini-batches** de datos, lo que reduce el costo computacional y puede mejorar la generalización.

## 🎯 Objetivo del notebook
1. Reutilizar el **Feature Engineering** y el **preprocesamiento** ya definidos en experimentos anteriores.  
2. Entrenar un pipeline con `SGDClassifier`.  
3. Evaluar con validación cruzada (5-fold stratified).  
4. Probar distintas configuraciones de `loss` y `penalty` para ver cuál generaliza mejor en Titanic.  
5. Generar un `submission.csv` y registrar el resultado.  


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 📥 Celda 2 — Carga de datos
En esta celda vamos a:  
- Importar las librerías necesarias (`pandas` para manipulación tabular).  
- Leer los datasets oficiales de Kaggle (`train.csv` y `test.csv`) desde la carpeta `/kaggle/input/titanic/`.  
- Verificar las dimensiones de ambos datasets para asegurarnos de que tengan el tamaño esperado:  
  - `train.csv` → 891 filas, 12 columnas (incluye la variable objetivo `Survived`).  
  - `test.csv` → 418 filas, 11 columnas (sin `Survived`, usado para predicción).  
- Mostrar las primeras filas del dataset de entrenamiento para inspeccionar la estructura y tipos de variables.  


In [2]:
# Importamos pandas para trabajar con datos en formato tabla (DataFrame)
import pandas as pd  

# Leemos el dataset de entrenamiento, que incluye la columna objetivo "Survived"
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")  

# Leemos el dataset de test, que no incluye "Survived" (lo vamos a predecir)
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")   

# Imprimimos la forma (filas, columnas) del train para validar tamaño esperado (891, 12)
print("Shape train:", train_df.shape)  

# Imprimimos la forma del test para validar tamaño esperado (418, 11)
print("Shape test :", test_df.shape)   

# Mostramos las primeras 5 filas del train para visualizar estructura, columnas y tipos de datos
train_df.head()  


Shape train: (891, 12)
Shape test : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 🔎 Celda 3 — Exploración inicial (EDA breve)
En esta celda vamos a:  
- Listar todas las columnas presentes en el dataset de entrenamiento.  
- Revisar valores faltantes en `train.csv` y `test.csv` (conteo y porcentaje).  
- Analizar la distribución de la variable objetivo `Survived` para ver el balance de clases.  

Esto nos permitirá identificar qué variables necesitan imputación y verificar si hay desbalance que pueda afectar al entrenamiento del modelo.


In [3]:
import numpy as np  # librería para cálculos numéricos

# --- Listado de columnas ---
print("Columnas en train:", train_df.columns.tolist())  # muestra todas las columnas de train

# --- Faltantes en TRAIN ---
na_train = train_df.isna().sum()                          # cuenta los valores NaN por columna
na_train_pct = (na_train / len(train_df)).round(3)        # porcentaje de NaN por columna
faltantes_train = pd.DataFrame({"nulos": na_train, "porcentaje": na_train_pct})  # tabla resumen
print("\nFaltantes en train (conteo y %):")
print(faltantes_train.sort_values("nulos", ascending=False))  # ordena de mayor a menor

# --- Faltantes en TEST ---
na_test = test_df.isna().sum()                            # cuenta los NaN en test
na_test_pct = (na_test / len(test_df)).round(3)           # porcentaje en test
faltantes_test = pd.DataFrame({"nulos": na_test, "porcentaje": na_test_pct})  # tabla resumen
print("\nFaltantes en test (conteo y %):")
print(faltantes_test.sort_values("nulos", ascending=False))

# --- Distribución de la variable objetivo (Survived) ---
y_counts = train_df["Survived"].value_counts().sort_index()               # conteo por clase (0 y 1)
y_ratio = train_df["Survived"].value_counts(normalize=True).round(4)      # proporción relativa
print("\nDistribución de Survived (conteo):")
print(y_counts)
print("\nDistribución de Survived (proporción):")
print(y_ratio)


Columnas en train: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Faltantes en train (conteo y %):
             nulos  porcentaje
Cabin          687       0.771
Age            177       0.199
Embarked         2       0.002
PassengerId      0       0.000
Survived         0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Fare             0       0.000

Faltantes en test (conteo y %):
             nulos  porcentaje
Cabin          327       0.782
Age             86       0.206
Fare             1       0.002
PassengerId      0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Embarked         0       0.000

Distribución de Survi

# 🧱 Celda 4 — Definición de variables base y objetivo
En esta celda vamos a:  
- Seleccionar las columnas **numéricas** y **categóricas** originales que servirán como punto de partida.  
- Construir la matriz de features `X` (entrenamiento) y `X_test` (predicción).  
- Definir la variable objetivo `y` (`Survived`).  
- Guardar `PassengerId` del test para generar el `submission.csv` al final.


In [4]:
# --- Definición de columnas base ---
# Variables numéricas originales del dataset
base_num_features = ["Age", "SibSp", "Parch", "Fare", "Pclass"]

# Variables categóricas originales del dataset
base_cat_features = ["Sex", "Embarked"]

# --- Construcción de X (features de entrenamiento) ---
X = train_df[base_num_features + base_cat_features].copy()  # seleccionamos columnas base del train

# --- Variable objetivo (target) ---
y = train_df["Survived"].astype(int)  # convertimos Survived a entero (0/1)

# --- Construcción de X_test (features de predicción) ---
X_test = test_df[base_num_features + base_cat_features].copy()  # seleccionamos columnas base del test

# --- Guardamos PassengerId (para submission) ---
test_passenger_id = test_df["PassengerId"].copy()  # IDs del test

# Vista previa de X
X.head()


,Age,SibSp,Parch,Fare,Pclass,Sex,Embarked
0,22.0,1,0,7.2500,3,male,S
1,38.0,1,0,71.2833,1,female,C
2,26.0,0,0,7.9250,3,female,S
3,35.0,1,0,53.1000,1,female,S
4,35.0,0,0,8.0500,3,male,S


# 🧪 Celda 5 — Ingeniería de atributos
En esta celda añadiremos nuevas variables derivadas a los datos:  

- `FamilySize = SibSp + Parch + 1`  
- `IsAlone = 1 si FamilySize == 1, en caso contrario 0`  
- `Title` (extraído de `Name`, normalizando y agrupando raros en `Other`)  
- `CabinKnown = 1 si la variable Cabin no es NaN`  
- `FarePerPerson = Fare / FamilySize`  
- `TicketGroupSize = número de pasajeros que comparten el mismo Ticket`  

Implementaremos estas transformaciones dentro de una función y la integraremos al pipeline usando `FunctionTransformer`.


In [5]:
from sklearn.preprocessing import FunctionTransformer  # para aplicar funciones custom en un Pipeline

# Definimos la función que creará las nuevas features
def add_features(df):
    df = df.copy()  # copiamos para no modificar el DataFrame original

    # --- FamilySize e IsAlone ---
    df["FamilySize"] = df["SibSp"].fillna(0) + df["Parch"].fillna(0) + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

    # --- Title (extraído de Name) ---
    if "Name" in df.columns:
        titles = df["Name"].str.extract(r",\s*([^\.]+)\.")[0]
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        titles = titles.where(titles.isin(["Mr","Mrs","Miss","Master"]), "Other")
        df["Title"] = titles.fillna("Other")
    else:
        df["Title"] = "Other"

    # --- CabinKnown ---
    if "Cabin" in df.columns:
        df["CabinKnown"] = (~df["Cabin"].isna()).astype(int)
    else:
        df["CabinKnown"] = 0

    # --- FarePerPerson ---
    df["FarePerPerson"] = df["Fare"].fillna(df["Fare"].median()) / df["FamilySize"].replace(0, 1)

    # --- TicketGroupSize ---
    if "Ticket" in df.columns:
        counts = df["Ticket"].map(df["Ticket"].value_counts())
        df["TicketGroupSize"] = counts.fillna(1).astype(int)
    else:
        df["TicketGroupSize"] = 1

    return df

# Creamos el transformador que usaremos en el Pipeline
feature_engineering = FunctionTransformer(add_features, validate=False)

# Añadimos columnas auxiliares a X y X_test necesarias para la ingeniería de atributos
for col in ["Name", "Ticket", "Cabin"]:
    if col not in X.columns and col in train_df.columns:
        X[col] = train_df[col]
    if col not in X_test.columns and col in test_df.columns:
        X_test[col] = test_df[col]

print("✅ Ingeniería de atributos lista (FunctionTransformer definido y columnas auxiliares añadidas).")


✅ Ingeniería de atributos lista (FunctionTransformer definido y columnas auxiliares añadidas).


# 🧽 Celda 6 — Preprocesamiento (imputación, escalado y One-Hot)
En esta celda definimos un `ColumnTransformer` que:
- Imputa **numéricas** con **mediana** y luego **escala** (SGD es sensible a magnitudes).
- Imputa **categóricas** con **moda** y aplica **One-Hot** (incluye `Title` creado en la Celda 5).
Solo listamos las **features finales** (las columnas auxiliares `Name`, `Ticket`, `Cabin` no se usan como entrada del modelo).


In [6]:
# Importamos utilidades para construir el preprocesamiento por tipo de columna
from sklearn.compose import ColumnTransformer            # permite aplicar distintos transformadores por columna
from sklearn.pipeline import Pipeline                    # para encadenar pasos (FE -> pre -> modelo)
from sklearn.impute import SimpleImputer                 # imputación de valores faltantes
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # escalado y one-hot

# Definimos las columnas que EXISTIRÁN DESPUÉS del Feature Engineering (Celda 5)
num_features = [                                         # variables numéricas finales
    "Age", "SibSp", "Parch", "Fare", "Pclass",           # numéricas originales
    "FamilySize", "IsAlone", "FarePerPerson",            # features creadas
    "TicketGroupSize", "CabinKnown"                      # features creadas (CabinKnown es binaria pero tratamos como num)
]
cat_features = [                                         # variables categóricas finales
    "Sex", "Embarked", "Title"                           # Title proviene del FE
]

# Pipeline para numéricas: imputar con mediana y escalar (muy importante para SGD)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),       # rellena NaN con la mediana (robusto a outliers)
    ("scaler", StandardScaler())                         # estandariza (media 0, desvío 1) para mejorar la optimización
])

# Pipeline para categóricas: imputar con moda y codificar con One-Hot
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),             # rellena NaN con el valor más frecuente
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # convierte a dummies densos
])

# Componemos el ColumnTransformer que aplicará cada pipeline a su grupo de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),      # aplica el pipeline numérico a num_features
        ("cat", categorical_transformer, cat_features)   # aplica el pipeline categórico a cat_features
    ],
    remainder="drop"                                     # descarta columnas no listadas (Name/Ticket/Cabin no pasan)
)

# Mensaje de control para verificar configuración
print("✅ Preprocesamiento definido para SGD.")
print("Num features:", num_features)
print("Cat features:", cat_features)


✅ Preprocesamiento definido para SGD.
Num features: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize', 'IsAlone', 'FarePerPerson', 'TicketGroupSize', 'CabinKnown']
Cat features: ['Sex', 'Embarked', 'Title']


# 🤖 Celda 7 — Pipeline completo (FE → Preprocesamiento → SGDClassifier) + CV
Entrenamos un **SGDClassifier** (modelo lineal entrenado con descenso de gradiente estocástico) usando:
- `loss="log_loss"` (equivalente a regresión logística),
- `penalty="elasticnet"` (mezcla de L1/L2),  
- validación cruzada **estratificada** de 5 folds con métrica *accuracy*.

> Nota: Este es un baseline razonable para SGD. En la próxima celda haremos un **GridSearch** con varias combinaciones (`loss`, `alpha`, `penalty`, `l1_ratio`).


In [7]:
# Importamos el clasificador y utilidades de validación
from sklearn.linear_model import SGDClassifier              # modelo lineal con SGD
from sklearn.model_selection import StratifiedKFold, cross_val_score  # CV estratificada y evaluación
from sklearn.pipeline import Pipeline                       # para encadenar pasos
import numpy as np                                          # utilidades numéricas

# Definimos un SGDClassifier "baseline" con configuración estable
sgd = SGDClassifier(
    loss="log_loss",     # pérdida tipo regresión logística (probabilística)
    penalty="elasticnet",# regularización combinada L1+L2 (controlada por l1_ratio)
    alpha=5e-4,          # fuerza de regularización (más alto = más regularización)
    l1_ratio=0.15,       # proporción de L1 dentro de elasticnet (0 = L2 puro, 1 = L1 puro)
    max_iter=2000,       # épocas máximas (iteraciones) para converger
    tol=1e-3,            # criterio de parada temprana (si mejora < tol)
    random_state=42      # reproducibilidad
    # class_weight="balanced"  # <- opcional si quisieras compensar el leve desbalance
)

# Armamos el pipeline completo: FE -> Preprocesamiento -> Modelo
pipe_sgd = Pipeline(steps=[
    ("fe", feature_engineering),  # agrega features creadas en la Celda 5
    ("pre", preprocessor),        # imputación + escalado + one-hot
    ("sgd", sgd)                  # clasificador lineal con SGD
])

# Definimos validación cruzada estratificada (mantiene proporción de clases 0/1 en cada fold)
cv = StratifiedKFold(
    n_splits=5,    # número de folds
    shuffle=True,  # mezclamos antes de partir
    random_state=42
)

# Ejecutamos cross-validation con accuracy (métrica oficial de la competencia)
cv_scores = cross_val_score(
    estimator=pipe_sgd,  # pipeline completo
    X=X,                 # features base (FE se aplica dentro del pipeline)
    y=y,                 # objetivo (Survived)
    cv=cv,               # esquema de validación
    scoring="accuracy",  # métrica de evaluación
    n_jobs=-1            # usa todos los núcleos disponibles
)

# Mostramos resultados por fold y resumen
print("Accuracy por fold:", np.round(cv_scores, 4))                 # muestra accuracy de cada fold
print("Accuracy promedio (CV):", cv_scores.mean().round(4),         # promedio
      "| Desv. std:", cv_scores.std().round(4))                     # dispersión entre folds


Accuracy por fold: [0.8268 0.8146 0.7809 0.8202 0.8483]
Accuracy promedio (CV): 0.8182 | Desv. std: 0.0219


# 🧪 Celda 8 — GridSearchCV para SGDClassifier
Buscamos la mejor combinación de hiperparámetros para **SGDClassifier**.  
Exploraremos:
- `loss` → `"log_loss"` (logística), `"hinge"` (SVM lineal), `"modified_huber"` (robusta).  
- `penalty` → `"l2"` (estable) y `"elasticnet"` (mezcla L1/L2).  
- `alpha` → fuerza de regularización (más alto = más regularización).  
- `l1_ratio` → proporción L1 dentro de `elasticnet` (solo aplica si `penalty="elasticnet"`).

Usamos **early stopping** dentro del estimador para estabilizar el entrenamiento estocástico.


In [8]:
# Importamos utilidades de búsqueda y validación
from sklearn.linear_model import SGDClassifier                    # clasificador lineal con SGD
from sklearn.model_selection import StratifiedKFold, GridSearchCV # validación cruzada y grid search
from sklearn.pipeline import Pipeline                             # para definir el pipeline completo
import numpy as np                                                # para listas de hiperparámetros
import pandas as pd                                               # para mostrar resultados ordenados

# 1) Definimos un SGDClassifier base (los hiperparámetros variarán en la grilla)
sgd_base = SGDClassifier(
    max_iter=5000,          # más iteraciones para asegurar convergencia
    tol=1e-3,               # criterio de parada
    early_stopping=True,    # activa parada temprana con un hold-out interno
    validation_fraction=0.1,# 10% del fold para validación interna del early stopping
    n_iter_no_change=5,     # paciencia para early stopping
    random_state=42,        # reproducibilidad
    # note: class_weight no está disponible en SGDClassifier
)

# 2) Definimos el pipeline completo: FE -> Preprocesamiento -> SGD
pipe_sgd = Pipeline(steps=[
    ("fe", feature_engineering),   # crea features (FamilySize, Title, etc.)
    ("pre", preprocessor),         # imputación + escalado + one-hot
    ("sgd", sgd_base)              # modelo a tunear
])

# 3) Definimos la grilla de hiperparámetros
alpha_values = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]    # regularización en escala log
l1_values    = [0.15, 0.3, 0.5]                  # solo aplican si penalty='elasticnet'

param_grid = [
    # penalty L2 (más estable)
    {
        "sgd__loss": ["log_loss", "modified_huber", "hinge"],
        "sgd__penalty": ["l2"],
        "sgd__alpha": alpha_values,
    },
    # penalty ElasticNet (mezcla L1/L2)
    {
        "sgd__loss": ["log_loss", "modified_huber", "hinge"],
        "sgd__penalty": ["elasticnet"],
        "sgd__alpha": alpha_values,
        "sgd__l1_ratio": l1_values,
    },
]

# 4) Esquema de validación cruzada estratificada (mantiene proporción 0/1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 5) Configuramos y ejecutamos el GridSearchCV
grid = GridSearchCV(
    estimator=pipe_sgd,        # pipeline completo
    param_grid=param_grid,     # combinaciones a explorar
    scoring="accuracy",        # métrica oficial de la competición
    cv=cv,                     # validación cruzada
    n_jobs=-1,                 # usa todos los cores disponibles
    refit=True,                # reentrena el mejor modelo en todo el train
    verbose=1                  # nivel de verbosidad
)

grid.fit(X, y)                 # entrena y evalúa todas las combinaciones

# 6) Reportamos el mejor resultado
print("🔝 Mejor accuracy (CV):", grid.best_score_.round(4))
print("🔧 Mejores hiperparámetros:", grid.best_params_)

# Guardamos el mejor pipeline ya ajustado
best_model = grid.best_estimator_

# 7) (Opcional) Mostramos el Top-5 de resultados ordenados por mean_test_score
res = pd.DataFrame(grid.cv_results_).sort_values("mean_test_score", ascending=False)
cols = [
    "mean_test_score","std_test_score",
    "param_sgd__loss","param_sgd__penalty","param_sgd__alpha","param_sgd__l1_ratio"
]
display(res[cols].head(5))


Fitting 5 folds for each of 60 candidates, totalling 300 fits
🔝 Mejor accuracy (CV): 0.8215
🔧 Mejores hiperparámetros: {'sgd__alpha': 0.01, 'sgd__loss': 'hinge', 'sgd__penalty': 'l2'}


,mean_test_score,std_test_score,param_sgd__loss,param_sgd__penalty,param_sgd__alpha,param_sgd__l1_ratio
14,0.821537,0.013158,hinge,l2,0.01,NaN
12,0.821537,0.007714,log_loss,l2,0.01,NaN
9,0.819302,0.012022,log_loss,l2,0.003,NaN
42,0.819296,0.004522,log_loss,elasticnet,0.003,0.15
53,0.818178,0.008471,hinge,elasticnet,0.01,0.15


# 📤 Celda 9 — Entrenamiento final y creación de `submission.csv`
Usamos el **mejor pipeline del GridSearch** (`best_model`) para predecir en `test` y
guardamos el archivo `submission.csv` con el formato requerido por Kaggle:

- Columnas: `PassengerId`, `Survived`
- Filas: 418 (una por cada pasajero del set de test)


In [9]:
# 1) Asegúrate de haber ejecutado la Celda 8 (GridSearch) para tener `best_model` en memoria.
#    `best_model` ya está reentrenado con refit=True sobre TODO el train del fold (pipeline completo).

# 2) Predicción en el set de test
test_preds = best_model.predict(X_test)      # obtenemos predicciones binarias 0/1
test_preds = test_preds.astype(int)          # convertimos explícitamente a enteros

# 3) Construcción del DataFrame de submission
submission = pd.DataFrame({
    "PassengerId": test_passenger_id,        # IDs del set de test
    "Survived": test_preds                   # predicciones del mejor modelo
})

# 4) Chequeos de formato (defensivos)
assert submission.shape[0] == 418, "El submission debe tener exactamente 418 filas."
assert list(submission.columns) == ["PassengerId", "Survived"], "Las columnas deben ser PassengerId y Survived."

# 5) Guardamos el CSV final para subir a Kaggle
submission.to_csv("submission.csv", index=False)

# 6) Confirmación y vista previa
print("✅ Archivo 'submission.csv' creado con", submission.shape[0], "filas.")
print(submission.head())
print("\nConteo predicciones:", submission['Survived'].value_counts().to_dict())


✅ Archivo 'submission.csv' creado con 418 filas.
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1

Conteo predicciones: {0: 256, 1: 162}


# 🏆 Celda 10 — Registro del resultado

- **Score Kaggle:** 0.77511  
- **Notebook:** EPA_TITANIC_SGD_V1.ipynb  
- **Modelo:** SGDClassifier (SVM lineal con descenso de gradiente estocástico)  
- **Mejores hiperparámetros (CV):**  
  - loss = "hinge"  
  - penalty = "l2"  
  - alpha = 0.01  
- **Resultados:**  
  - CV promedio: 0.8215  
  - Kaggle score: 0.77511 (**mejor modelo hasta ahora**)  
- **Notas:**  
  - La regularización relativamente fuerte (`alpha=0.01`) estabilizó el modelo.  
  - El modelo replica muy bien la proporción real de clases.  
  - Próximos pasos: probar ensambles (Random Forest, Gradient Boosting) para intentar superar 0.78+.  
